# IT Company Project — Documentation & Development Notebook

This notebook documents the `it-Ideatask` project, setup steps for running Java/Gradle/JUnit/Mockito, SonarQube integration notes, example code snippets from the project, and simple unit-test examples (Mockito where relevant).

**Environment**
- OS: Windows (confirmed)
- Project root: `C:\Users\danie\Documents\Solvd\it-Ideatask`
- JDK: 21
- Recommended: IntelliJ IDEA (already used), Jupyter (for viewing this notebook)


----
## 1) Quick setup checklist

1. Ensure JDK 21 is installed and `java`/`javac` are on PATH.
2. Install Gradle wrapper or use the project's `gradlew` (Windows: `.\gradlew`).
3. Run tests from the project root with Gradle:
   ```powershell
   .\gradlew test
   ```
4. If you want Java support in Jupyter, consider installing IJava (instructions below).
5. SonarQube: have a running SonarQube server (local or remote) and `sonar-scanner` installed or configure scanner in Gradle/Maven.


----
## 2) IJava kernel (Jupyter Java kernel) 

Cloned `SpencerPark/IJava`. Typical install steps on Windows:
1. Build the project with Gradle (from the IJava directory):
   ```powershell
   .\gradlew install
   ```
2. If `build.gradle` or `buildSrc` uses old Gradle features, you may need Gradle wrapper to pick compatible version. Use `--refresh-dependencies` if needed.
3. After successful build, a kernel spec is installed and `jupyter kernelspec list` should include `java`.

If building `IJava` fails with Gradle plugin errors (e.g. `compile()` method not found in modern Gradle), try opening `buildSrc/build.gradle` and change `compile` to `implementation` or use a Gradle version compatible with the project. Be careful — modifying the kernel build scripts may be needed.

Alternatively, consider using VS Code with Java extensions or IntelliJ's built-in notebook features instead of a Java kernel in Jupyter.

----
## 3) SonarQube integration (high-level)

Add SonarQube scanning to Gradle (example `build.gradle` snippet):
```groovy
plugins {
  id "org.sonarqube" version "4.3.0.3225"
}
sonarqube {
  properties {
    property "sonar.projectKey", "your-key"
    property "sonar.host.url", "http://localhost:9000"
    property "sonar.login", "YOUR_TOKEN"
  }
}
```

Run from project root:
```powershell
.\gradlew sonarqube
```

If you cannot run `sonarqube` from Gradle, you can use `sonar-scanner` CLI and configure `sonar-project.properties` with the same properties.

----
## 4) Project structure & important classes

Below are important classes from the project and brief explanation:
- `Report` — a simple container for cost/duration/people; annotated with `@CheckBeforeDelivery` and extends `Procedures`.
- `ProjectProcess` — holds timeline info and evaluates deadline achievability.
- DAO implementations (e.g., `SplunkMonitoringDAOImpl`) — perform DB operations.

### `Report.java`  — concise review
```java
public class Report extends Procedures {
  private float totalCost; // ...
  private int totalDays;
  private int totalPeople;
  public Report(float totalCost, int totalDays, int totalPeople, int timeOffScheduled) {
    super(timeOffScheduled);
    this.totalCost = totalCost;
    this.totalDays = totalDays;
    this.totalPeople = totalPeople;
  }
  // getters, setters, printReport(), equals/hashCode
}
```


----
## 5) Unit testing: example with JUnit + Mockito

We'll create a minimal unit test for `Report` that *does not* try to inject or mock the `static final Logger` field (that caused errors). Instead, test behaviour of getters, setters, equals/hashCode and `printReport()` doesn't throw.

Add these dependencies to your Gradle `build.gradle` (or Maven equivalent):
```groovy
dependencies {
  testImplementation 'org.junit.jupiter:junit-jupiter-api:5.11.0'
  testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.11.0'
  testImplementation 'org.mockito:mockito-core:5.11.0'
}
test {
  useJUnitPlatform()
}
```

### Example test
```java
package com.solvd.it.company;
import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;
class ReportTest {
  @Test
  void gettersSettersAndEqualsHashCode() {
    Report a = new Report(10000f, 30, 5, 2);
    Report b = new Report(10000f, 30, 5, 2);
    assertEquals(10000f, a.getTotalCost());
    assertEquals(30, a.getTotalDays());
    assertEquals(5, a.getTotalPeople());
    assertEquals(a, b);
    assertEquals(a.hashCode(), b.hashCode());
  }
  @Test
  void printReportDoesNotThrow() {
    Report r = new Report(5000f, 15, 3, 1);
    assertDoesNotThrow(() -> r.printReport());
  }
}
```

This avoids mocking `LOGGER` (static final) and focuses on object behaviour which is easy and robust to test.

----
## 6) Troubleshooting: logger mocking & static final fields

- Java `static final` fields cannot be reassigned via reflection on modern JVMs (access restrictions). Instead:
  - Avoid mocking the `Logger` directly; test behaviour rather than log output.
  - If you absolutely must capture logs, configure Log4j to write to an in-memory appender or file and read from it; or use Log4j test utilities.
- For `NoSuchElementException` during tests: ensure any `Scanner` reads are fed via test input or remove interactive prompts for unit tests.
- For `InputMismatchException` or parsing errors: ensure parsing calls use correct formats (e.g., `LocalDate.parse(..., DateTimeFormatter.ofPattern("MM/dd/yyyy"))`).


----
## 7) DB DAO notes (`SplunkMonitoringDAOImpl`)

- Created `save` method correctly with requests generated keys. The DB exception earlier (`Duplicate entry '101' for key ...`) indicates: your test data attempted to insert a record with a primary key that already exists (or `project_code` is unique).
- Fixes:
  1. Ensure the table's primary key is an auto-increment `id` (and you're not inserting it manually), and that `project_code` is not marked `PRIMARY` unless you expect unique codes.
  2. In tests, either use a dedicated test database schema and clear it before tests, or mock the DAO's database interactions with Mockito.

Example mock usage for DAO:
```java
SplunkMonitoringDAO dao = mock(SplunkMonitoringDAO.class);
when(dao.save(any())).thenReturn(123);
int id = dao.save(new SplunkMonitoring(...));
assertEquals(123, id);
```


----
## 8) Useful commands (Windows PowerShell)

Build project:
```powershell
.\gradlew build --refresh-dependencies
.\gradlew test
```
Run SonarQube (if configured):
```powershell
.\gradlew sonarqube
```
Run a single test class:
```powershell
.\gradlew test --tests com.solvd.it.company.ReportTest
```


----
## 9) Full example: simple Mockito test to validate `Report` equals logic

```java
import static org.mockito.Mockito.*;
import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;

class ReportMockitoTest {
  @Test
  void exampleMockedReportConstruction() {
    // We don't need to mock Report, but show a simple Mockito usage
    Report r = spy(new Report(1f,1,1,0));
    doReturn(42).when(r).getTotalDays();
    assertEquals(42, r.getTotalDays());
  }
}
```
